## Import libraries:

In [27]:
from imports import *

## Detector map:

The idea is to create a matrix map of the detector. Currently we have taken two approaches: 

1 - realistic representation 2 - compressed representation (schematic)

### 1 - Realistic representation:

In [2]:
import math

def detector_matrix(data_dict, filter_labels, pixel):
    """
    Create a matrix from the data dictionary, filter by labels, and normalize coordinates.

    Parameters:
    - data_dict: Dictionary containing data with y, z coordinates and labels.
    - filter_labels: Set of labels to filter by.
    - pixel: Tuple (width, height) used to calculate the size of the matrix.
    
    Returns:
    - Realistic matrix of the detector
    """
    import math
    
    # Calculate size of the matrix based on pixel dimensions
    size = (math.ceil(400 / pixel[0])+1, math.ceil(500 / pixel[1]))

    # Extract y and z values and the corresponding indices from the dictionary
    y_values = [val[1] for val in data_dict.values()]
    z_values = [val[2] for val in data_dict.values()]
    labels = [val[3] for val in data_dict.values()]
    indices = list(data_dict.keys())

    # Normalize and scale the y and z data to fit into the matrix
    y_min, y_max, z_min, z_max = -200, 200, 0, 500
    y_scaled = [(y - y_min) / (y_max - y_min) * (size[0] - 1) for y in y_values]
    z_scaled = [(z - z_min) / (z_max - z_min) * (size[1] - 1) for z in z_values]

    # Rounding to the nearest integer for matrix indices
    y_indices = [min(max(round(y), 0), size[0] - 1) for y in y_scaled]  # Ensure indices are within bounds
    z_indices = [min(max(round(z), 0), size[1] - 1) for z in z_scaled]  # Ensure indices are within bounds

    # Create the matrix and initialize it with -2 for unoccupied cells
    matrix = np.full(size, -2)

    # Populate the matrix with the indices from the dictionary
    for idx, (y, z) in enumerate(zip(y_indices, z_indices)):
        if labels[idx] in filter_labels:
            matrix[y, z] = indices[idx]

    return matrix


#correccion de aprox. 
def correccion(matrix):
    row_toinsert = np.full((1, matrix.shape[1]), -2)
    column_toinsert = np.full((matrix.shape[0]+4, 1), -2)
    row_modified_matrix = np.insert(matrix, [12, 24, 32, 42], row_toinsert, axis=0)
    total_modified_matrix = np.insert(row_modified_matrix, [17,34,matrix.shape[1]], column_toinsert, axis=1)
    return total_modified_matrix

import numpy as np

def create_realistic_maps(even_data_dict, pixel):
    # Define the configurations for each map
    map_configs = {
        'coated_pmt_map_realistic': {0},
        'uncoated_pmt_map_realistic': {1},
        'xarap_vuv_map_realistic': {2},
        'xarap_vis_map_realistic': {3},
        'pmts_map_realistic': {0, 1},
        'xarap_map_realistic': {2, 3},
        'vis_map_realistic': {0, 1, 3},
        'vuv_map_realistic': {0, 2},
        'total_map_realistic': {0, 1, 2, 3}
    }

    # Create each map and save it
    for map_name, detector_set in map_configs.items():
        # Process the detector matrix and apply correction
        map_data = correccion(detector_matrix(even_data_dict, detector_set, pixel))
        
        # Stack the data and save to CSV
        map_data = np.row_stack((map_data, map_data + 1))
        np.savetxt(f'saved_archives/maps/realistic/{map_name}.csv', map_data, fmt='%d', delimiter=',')

# Example usage
# create_realistic_maps(even_data_dict, pixel)


In [4]:
file_path = '/data/vidales/opana_tree_combined_v0411_both.root'
file = uproot.open(file_path)
PDSMap = file['opanatree']['PDSMapTree']

In [6]:
ID = PDSMap['OpDetID'].array()
X = PDSMap['OpDetX'].array()
Y = PDSMap['OpDetY'].array()
Z = PDSMap['OpDetZ'].array()
Type = PDSMap['OpDetType'].array()

# Make dictionaries with the data
data_dict = {id_val: (x_val, y_val, z_val,int(type_val)) for id_val, x_val, y_val, z_val, type_val in zip(ID[0], X[0], Y[0], Z[0],Type[0])}
even_data_dict = {id_val: data for id_val, data in data_dict.items() if id_val % 2 == 0}
odd_data_dict = {id_val: data for id_val, data in data_dict.items() if id_val % 2 != 0}

In [ ]:
# Define the pixel size in cm
pixel = (7.5,7.5)


# Create the detector matrices

map_configs = {
    'coated_pmt_map_realistic': {0},
    'uncoated_pmt_map_realistic': {1},
    'xarap_vuv_map_realistic': {2},
    'xarap_vis_map_realistic': {3},
    'pmts_map_realistic': {0, 1},
    'xarap_map_realistic': {2, 3},
    'vis_map_realistic': {0, 1, 3},
    'vuv_map_realistic': {0, 2},
    'total_map_realistic': {0, 1, 2, 3}
}

# Create each map and save it
for map_name, detector_set in map_configs.items():
    # Process the detector matrix and apply correction
    map_data = correccion(detector_matrix(even_data_dict, detector_set, pixel))
    
    # Stack the data and save to CSV
    map_data = np.row_stack((map_data, map_data + 1))
    np.savetxt(f'saved_archives/maps/realistic/{map_name}.csv', map_data, fmt='%d', delimiter=',')

### 2 - Schematic representation:

In [19]:
coated_pmt_map_compressed = np.array([[-2, 6, -2, -2, -2, -2, -2, 60, -2, -2, 84, -2, -2, -2, -2, -2, 138, -2, -2, 162, -2, -2, -2, -2, -2, 216, -2, -2, 240, -2, -2, -2, -2, -2, 294, -2], 
                                   [-2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2], 
                                   [-2, 8, -2, -2, -2, -2, -2, 62, -2, -2, 86, -2, -2, -2, -2, -2, 140, -2, -2, 164, -2, -2, -2, -2, -2, 218, -2, -2, 242, -2, -2, -2, -2, -2, 296, -2], 
                                   [-2, 10, -2, -2, -2, -2, -2, 64, -2, -2, 88, -2, -2, -2, -2, -2, 142, -2, -2, 166, -2, -2, -2, -2, -2, 220, -2, -2, 244, -2, -2, -2, -2, -2, 298, -2], 
                                   [-2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2], 
                                   [-2, 12, -2, -2, -2, -2, -2, 66, -2, -2, 90, -2, -2, -2, -2, -2, 144, -2, -2, 168, -2, -2, -2, -2, -2, 222, -2, -2, 246, -2, -2, -2, -2, -2, 300, -2], 
                                   [-2, 14, -2, -2, -2, -2, -2, 68, -2, -2, 92, -2, -2, -2, -2, -2, 146, -2, -2, 170, -2, -2, -2, -2, -2, 224, -2, -2, 248, -2, -2, -2, -2, -2, 302, -2], 
                                   [-2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2], 
                                   [-2, 16, -2, -2, -2, -2, -2, 70, -2, -2, 94, -2, -2, -2, -2, -2, 148, -2, -2, 172, -2, -2, -2, -2, -2, 226, -2, -2, 250, -2, -2, -2, -2, -2, 304, -2]])

uncoated_pmt_map_compressed = np.array([[-2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2], 
                                     [-2, -2, -2, -2, 36, -2, -2, -2, -2, -2, -2, -2, -2, 114, -2, -2, -2, -2, -2, -2, -2, -2, 192, -2, -2, -2, -2, -2, -2, -2, -2, 270, -2, -2, -2, -2], 
                                     [-2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2], 
                                     [-2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2], 
                                     [-2, -2, -2, -2, 38, -2, -2, -2, -2, -2, -2, -2, -2, 116, -2, -2, -2, -2, -2, -2, -2, -2, 194, -2, -2, -2, -2, -2, -2, -2, -2, 272, -2, -2, -2, -2], 
                                     [-2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2], 
                                     [-2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2], 
                                     [-2, -2, -2, -2, 40, -2, -2, -2, -2, -2, -2, -2, -2, 118, -2, -2, -2, -2, -2, -2, -2, -2, 196, -2, -2, -2, -2, -2, -2, -2, -2, 274, -2, -2, -2, -2], 
                                     [-2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2]])

xarap_vuv_map_compressed = np.array([[-2, -2, -2, -2, -2, 42, -2, -2, -2, -2, -2, -2, -2, -2, 120, -2, -2, -2, -2, -2, -2, -2, -2, 198, -2, -2, -2, -2, -2, -2, -2, -2, 276, -2, -2, -2],
                                  [-2, -2, 18, -2, -2, -2, -2, -2, 72, -2, -2, 96, -2, -2, -2, -2, -2, 150, -2, -2, 174, -2, -2, -2, -2, -2, 228, -2, -2, 252, -2, -2, -2, -2, -2, 306], 
                                  [-2, -2, -2, -2, -2, 44, -2, -2, -2, -2, -2, -2, -2, -2, 122, -2, -2, -2, -2, -2, -2, -2, -2, 200, -2, -2, -2, -2, -2, -2, -2, -2, 278, -2, -2, -2], 
                                  [-2, -2, -2, -2, -2, 46, -2, -2, -2, -2, -2, -2, -2, -2, 124, -2, -2, -2, -2, -2, -2, -2, -2, 202, -2, -2, -2, -2, -2, -2, -2, -2, 280, -2, -2, -2], 
                                  [-2, -2, 10, -2, -2, -2, -2, -2, 74, -2, -2, 98, -2, -2, -2, -2, -2, 152, -2, -2, 176, -2, -2, -2, -2, -2, 230, -2, -2, 254, -2, -2, -2, -2, -2, 308], 
                                  [-2, -2, -2, -2, -2, 48, -2, -2, -2, -2, -2, -2, -2, -2, 126, -2, -2, -2, -2, -2, -2, -2, -2, 204, -2, -2, -2, -2, -2, -2, -2, -2, 282, -2, -2, -2], 
                                  [-2, -2, -2, -2, -2, 50, -2, -2, -2, -2, -2, -2, -2, -2, 128, -2, -2, -2, -2, -2, -2, -2, -2, 206, -2, -2, -2, -2, -2, -2, -2, -2, 284, -2, -2, -2], 
                                  [-2, -2, 22, -2, -2, -2, -2, -2, 76, -2, -2, 100, -2, -2, -2, -2, -2, 154, -2, -2, 178, -2, -2, -2, -2, -2, 232, -2, -2, 256, -2, -2, -2, -2, -2, 310], 
                                  [-2, -2, -2, -2, -2, 52, -2, -2, -2, -2, -2, -2, -2, -2, 130, -2, -2, -2, -2, -2, -2, -2, -2, 208, -2, -2, -2, -2, -2, -2, -2, -2, 286, -2, -2, -2]])

xarap_vis_map_compressed = np.array([[-2, -2, -2, 24, -2, -2, -2, -2, -2, -2, -2, -2, 102, -2, -2, -2, -2, -2, -2, -2, -2, 180, -2, -2, -2, -2, -2, -2, -2, -2, 258, -2, -2, -2, -2, -2], 
                                  [0, -2, -2, -2, -2, -2, 54, -2, -2, 78, -2, -2, -2, -2, -2, 132, -2, -2, 156, -2, -2, -2, -2, -2, 210, -2, -2, 234, -2, -2, -2, -2, -2, 288, -2, -2], 
                                  [-2, -2, -2, 26, -2, -2, -2, -2, -2, -2, -2, -2, 104, -2, -2, -2, -2, -2, -2, -2, -2, 182, -2, -2, -2, -2, -2, -2, -2, -2, 260, -2, -2, -2, -2, -2], 
                                  [-2, -2, -2, 28, -2, -2, -2, -2, -2, -2, -2, -2, 106, -2, -2, -2, -2, -2, -2, -2, -2, 184, -2, -2, -2, -2, -2, -2, -2, -2, 262, -2, -2, -2, -2, -2], 
                                  [2, -2, -2, -2, -2, -2, 56, -2, -2, 80, -2, -2, -2, -2, -2, 134, -2, -2, 158, -2, -2, -2, -2, -2, 212, -2, -2, 236, -2, -2, -2, -2, -2, 290, -2, -2], 
                                  [-2, -2, -2, 30, -2, -2, -2, -2, -2, -2, -2, -2, 108, -2, -2, -2, -2, -2, -2, -2, -2, 186, -2, -2, -2, -2, -2, -2, -2, -2, 264, -2, -2, -2, -2, -2], 
                                  [-2, -2, -2, 32, -2, -2, -2, -2, -2, -2, -2, -2, 110, -2, -2, -2, -2, -2, -2, -2, -2, 188, -2, -2, -2, -2, -2, -2, -2, -2, 266, -2, -2, -2, -2, -2], 
                                  [4, -2, -2, -2, -2, -2, 58, -2, -2, 82, -2, -2, -2, -2, -2, 136, -2, -2, 160, -2, -2, -2, -2, -2, 214, -2, -2, 238, -2, -2, -2, -2, -2, 292, -2, -2], 
                                  [-2, -2, -2, 34, -2, -2, -2, -2, -2, -2, -2, -2, 112, -2, -2, -2, -2, -2, -2, -2, -2, 190, -2, -2, -2, -2, -2, -2, -2, -2, 268, -2, -2, -2, -2, -2]])


#plot total for prove
#total_map_compressed = coated_pmt_map_compressed +uncoated_pmt_map_compressed +xarap_vuv_map_compressed +xarap_vis_map_compressed
#plt.figure(figsize=(12, 12))
#plt.imshow(total_map_compressed, cmap='viridis', interpolation='none')

In [17]:
def custom_add_matrices(matrices):
    # Assume matrices is a list of numpy arrays with the same shape
    total_matrix = np.full_like(matrices[0], -2)  # Start with a matrix filled with -2

    for i in range(total_matrix.shape[0]):  # Loop through rows
        for j in range(total_matrix.shape[1]):  # Loop through columns
            values = [matrix[i, j] for matrix in matrices]
            # Check if all values are -2
            if all(value == -2 for value in values):
                total_matrix[i, j] = -2
            if all(value == -1 for value in values):
                total_matrix[i, j] = -1
            else:
                # Take the first value that is different from -2
                for value in values:
                    if (value != -2) & (value != -1) :
                        total_matrix[i, j] = value
                        break

    return total_matrix

vis map y vuv map se pueden realizar con el custom add matrices

In [20]:
# mapas individuales
coated_pmt_map_compressed  = np.row_stack((coated_pmt_map_compressed ,coated_pmt_map_compressed +1))
np.savetxt('saved_archives/maps/compressed/coated_pmt_map_compressed.csv', coated_pmt_map_compressed , fmt='%d', delimiter=',')
uncoated_pmt_map_compressed  = np.row_stack((uncoated_pmt_map_compressed ,uncoated_pmt_map_compressed +1))
np.savetxt('saved_archives/maps/compressed/uncoated_pmt_map_compressed.csv', uncoated_pmt_map_compressed , fmt='%d', delimiter=',')
xarap_vuv_map_compressed  = np.row_stack((xarap_vuv_map_compressed ,xarap_vuv_map_compressed +1))
np.savetxt('saved_archives/maps/compressed/xarap_vuv_map_compressed.csv', xarap_vuv_map_compressed , fmt='%d', delimiter=',')
xarap_vis_map_compressed  = np.row_stack((xarap_vis_map_compressed ,xarap_vis_map_compressed +1))
np.savetxt('saved_archives/maps/compressed/xarap_vis_map_compressed.csv', xarap_vis_map_compressed , fmt='%d', delimiter=',')

# mapa pmts
pmts_map_compressed = custom_add_matrices([coated_pmt_map_compressed,uncoated_pmt_map_compressed])
np.savetxt('saved_archives/maps/compressed/pmts_map_compressed.csv', pmts_map_compressed, fmt='%d', delimiter=',')

# mapa XAs
xarapucas_map_compressed = custom_add_matrices([xarap_vuv_map_compressed,xarap_vis_map_compressed])
np.savetxt('saved_archives/maps/compressed/xarapucas_map_compressed.csv', xarapucas_map_compressed, fmt='%d', delimiter=',')

# VIS/VUV
vis_map_compressed = custom_add_matrices([coated_pmt_map_compressed, uncoated_pmt_map_compressed, xarap_vis_map_compressed])
np.savetxt('saved_archives/maps/compressed/vis_map_compressed.csv', vis_map_compressed, fmt='%d', delimiter=',')
vuv_map_compressed = custom_add_matrices([coated_pmt_map_compressed, xarap_vuv_map_compressed])
np.savetxt('saved_archives/maps/compressed/vuv_map_compressed.csv', vuv_map_compressed, fmt='%d', delimiter=',')

#total
total_map_compressed = custom_add_matrices([coated_pmt_map_compressed,uncoated_pmt_map_compressed,xarap_vuv_map_compressed,xarap_vis_map_compressed])
np.savetxt('saved_archives/maps/compressed/total_map_compressed.csv', total_map_compressed, fmt='%d', delimiter=',')

### XAs maps from V2_4_2-Finer+4Time (Alberto)

In [2]:
vis_map = np.array(([-2,-2,24,-2,-2,-2,-2,-2,102,-2,-2,-2,-2,-2,180,-2,-2,-2,-2,-2,258,-2,-2,-2],
                           [0,-2,-2,-2,54,-2,78,-2,-2,-2,132,-2,156,-2,-2,-2,210,-2,234,-2,-2,-2,288,-2],
                           [-2,-2,26,-2,-2,-2,-2,-2,104,-2,-2,-2,-2,-2,182,-2,-2,-2,-2,-2,260,-2,-2,-2],
                           [-2,-2,28,-2,-2,-2,-2,-2,106,-2,-2,-2,-2,-2,184,-2,-2,-2,-2,-2,262,-2,-2,-2],
                           [2,-2,-2,-2,56,-2,80,-2,-2,-2,134,-2,158,-2,-2,-2,212,-2,236,-2,-2,-2,290,-2],
                           [-2,-2,30,-2,-2,-2,-2,-2,108,-2,-2,-2,-2,-2,186,-2,-2,-2,-2,-2,264,-2,-2,-2],
                           [-2,-2,32,-2,-2,-2,-2,-2,110,-2,-2,-2,-2,-2,188,-2,-2,-2,-2,-2,266,-2,-2,-2],
                           [4,-2,-2,-2,58,-2,82,-2,-2,-2,136,-2,160,-2,-2,-2,214,-2,238,-2,-2,-2,292,-2],
                           [-2,-2,34,-2,-2,-2,-2,-2,112,-2,-2,-2,-2,-2,190,-2,-2,-2,-2,-2,268,-2,-2,-2]))

vuv_map = np.array(([-2,-2,-2,42,-2,-2,-2,-2,-2,120,-2,-2,-2,-2,-2,198,-2,-2,-2,-2,-2,276,-2,-2],
                           [-2,18,-2,-2,-2,72,-2,96,-2,-2,-2,150,-2,174,-2,-2,-2,228,-2,252,-2,-2,-2,306],
                           [-2,-2,-2,44,-2,-2,-2,-2,-2,122,-2,-2,-2,-2,-2,200,-2,-2,-2,-2,-2,278,-2,-2],
                           [-2,-2,-2,46,-2,-2,-2,-2,-2,124,-2,-2,-2,-2,-2,202,-2,-2,-2,-2,-2,280,-2,-2],
                           [-2,10,-2,-2,-2,74,-2,98,-2,-2,-2,152,-2,176,-2,-2,-2,230,-2,254,-2,-2,-2,308],
                           [-2,-2,-2,48,-2,-2,-2,-2,-2,126,-2,-2,-2,-2,-2,204,-2,-2,-2,-2,-2,282,-2,-2],
                           [-2,-2,-2,50,-2,-2,-2,-2,-2,128,-2,-2,-2,-2,-2,206,-2,-2,-2,-2,-2,284,-2,-2],
                           [-2,22,-2,-2,-2,76,-2,100,-2,-2,-2,154,-2,178,-2,-2,-2,232,-2,256,-2,-2,-2,310],
                           [-2,-2,-2,52,-2,-2,-2,-2,-2,130,-2,-2,-2,-2,-2,208,-2,-2,-2,-2,-2,286,-2,-2]))

vis_map = np.row_stack((vis_map,vis_map+1))
vuv_map = np.row_stack((vuv_map,vuv_map+1))

np.savetxt('saved_archives/maps/Alberto_maps/alberto_vis_map.csv', vis_map , fmt='%d', delimiter=',')
np.savetxt('saved_archives/maps/Alberto_maps/alberto_vuv_map.csv', vuv_map , fmt='%d', delimiter=',')

# 2D and 3D detector location map

In [61]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots


# Extract coordinates, color values, and IDs for even and odd 2D plots
hor_even = [coord[2] for coord in even_data_dict.values()]
ver_even = [coord[1] for coord in even_data_dict.values()]
color_even = [coord[3] for coord in even_data_dict.values()]
ids_even = [key for key in even_data_dict.keys()]

hor_odd = [coord[2] for coord in odd_data_dict.values()]
ver_odd = [coord[1] for coord in odd_data_dict.values()]
color_odd = [coord[3] for coord in odd_data_dict.values()]
ids_odd = [key for key in odd_data_dict.keys()]
# Define a custom color map with distinct colors
distinct_colors = ['rgb(255,0,0)', 'rgb(0,255,0)', 'rgb(0,0,255)', 'rgb(255,0,255)']
color_map = {i: distinct_colors[i % len(distinct_colors)] for i in range(len(set(color_even)))}
colored_text_odd = [f'<span style="color: {color_map.get(col, "black")}">{id_}</span>' for id_, col in zip(ids_odd, color_odd)]
colored_text_even = [f'<span style="color: {color_map.get(col, "black")}">{id_}</span>' for id_, col in zip(ids_even, color_even)]

# Create subplot grid
fig = make_subplots(rows=2, cols=1,subplot_titles=('x>0, Odd IDs', 'x<0, Even IDs'), vertical_spacing=0.15)

# Add 2D scatter plot for even IDs
fig.add_trace(go.Scatter(
    x=hor_even,
    y=ver_even,
    mode='text',  # Use both markers and text
    text=colored_text_even,  # Display colored IDs as text labels
    textposition='top center',  # Position text labels
    marker=dict(
        size=10,  # Marker size
        color=color_even,  # Set color based on the fourth dimension
        colorscale='Jet',  # Define color scale
        colorbar=dict(title='Color Scale')  # Add colorbar to show color scale
    ),
    hoverinfo='text',  # Show text on hover
), row=2, col=1)

# Add 2D scatter plot for odd IDs
fig.add_trace(go.Scatter(
    x=hor_odd,
    y=ver_odd,
    mode='text',  # Use both markers and text
    text=colored_text_odd,  # Display colored IDs as text labels
    textposition='top center',  # Position text labels
    marker=dict(
        size=10,  # Marker size
        color=color_odd,  # Set color based on the fourth dimension
        colorscale='Jet',  # Define color scale
        colorbar=dict(title='Color Scale')  # Add colorbar to show color scale
    ),
    hoverinfo='text',  # Show text on hover
), row=1, col=1)

legend_labels = ['0: CoatedPMT (VIS and VUV)', '1: UncoatedPMT (only VIS)', '2: VUV XARAPUCA', '3: VIS XARAPUCA']
for i, label in enumerate(legend_labels):
    fig.add_trace(go.Scatter(
        x=[None],
        y=[None],
        mode='markers',
        marker=dict(
            size=10,
            color=distinct_colors[i],  # Color for each label
            showscale=False
        ),
        name=label
    ))

# Update layout
fig.update_layout(
    title='2D Detector Planes',
    width=1200,  # Set figure width
    height=800,  # Set figure height
    xaxis=dict(
        title='Z',
    ),
    yaxis=dict(
        title='Y',
    ),
    xaxis2=dict(
        title='Z',
    ),
    yaxis2=dict(
        title='Y',
    ),
    margin=dict(l=10, r=10, t=50, b=10),
    annotations=[dict(
    text=anno.text,
    x=anno.x,
    y=anno.y + 0.02,
    font=dict(size=25),  # Cambia el tamaño de la fuente
    showarrow=False
) for anno in fig.layout.annotations]
)

# Show the plot
fig.show()

Mapa 3D detectores

In [26]:
import plotly.graph_objs as go
import plotly.express as px

# Tu diccionario de datos
data = data_dict

# Extraer las coordenadas
x = [coord[0] for coord in data.values()]
y = [coord[1] for coord in data.values()]  # Ahora Y va al eje vertical
z = [coord[2] for coord in data.values()]  # Z va al eje horizontal
color = [coord[3] for coord in data.values()]
ids = [key for key in data_dict.keys()]

# Crear las etiquetas de hover con las coordenadas correctas
hover_text = [f'ID: {id_}<br>X: {x_}<br>Y: {y_}<br>Z: {z_:.1f}' for id_, x_, y_, z_ in zip(ids, x, y, z)]

# Crear el gráfico de dispersión 3D
fig = go.Figure(data=[go.Scatter3d(
    x=x,
    y=z,  # Y es el eje vertical
    z=y,  # Z es el eje horizontal
    mode='markers',
    marker=dict(
        size=5,
        color=color,  # Establece el color basado en la cuarta dimensión
        colorbar=dict(title='Color Scale')  # Añadir la barra de colores
    ),
    text=hover_text,  # Mostrar las coordenadas correctas en el hover
    hoverinfo='text'  # Mostrar solo el texto del hover
)])

# Establecer etiquetas de los ejes
fig.update_layout(scene=dict(
    xaxis_title='X',
    yaxis_title='Z',  # Y es ahora el eje vertical
    zaxis_title='Y'  # Z es ahora el eje horizontal
))

# Mostrar el gráfico
fig.show()


### Cambio en mapas actuales

ahora si que es equivalente a la imagen de 2D detector planes.

In [67]:
import numpy as np
import os

# Ruta base donde están los archivos originales
base_path = 'saved_archives/maps/realistic/non_flipped'

# Lista de nombres de archivos
filenames = [
    "coated_pmt_map_realistic.csv",
    "pmts_map_realistic.csv",
    "total_map_realistic.csv",
    "uncoated_pmt_map_realistic.csv",
    "vis_map_realistic.csv",
    "vuv_map_realistic.csv",
    "xarap_map_realistic.csv",
    "xarap_vis_map_realistic.csv",
    "xarap_vuv_map_realistic.csv"
]

# Crear una carpeta para guardar los archivos invertidos (opcional)
flipped_path = os.path.join(base_path, "flipped")
os.makedirs(flipped_path, exist_ok=True)

# Procesar y guardar cada archivo
for filename in filenames:
    filepath = os.path.join(base_path, filename)
    
    # Cargar el archivo
    matrix = np.loadtxt(filepath, delimiter=',', dtype=int)  # Asegura que se cargue como enteros
    
    # Invertir la matriz
    flipped_matrix = matrix[::-1]
    
    # Definir nuevo nombre de archivo
    flipped_filename = filename.replace(".csv", "_flipped.csv")
    flipped_filepath = os.path.join(flipped_path, flipped_filename)
    
    # Guardar el archivo invertido sin notación científica
    np.savetxt(flipped_filepath, flipped_matrix, delimiter=',', fmt='%d')

    print(f"Guardado: {flipped_filepath}")


Guardado: saved_archives/maps/realistic/non_flipped/flipped/coated_pmt_map_realistic_flipped.csv
Guardado: saved_archives/maps/realistic/non_flipped/flipped/pmts_map_realistic_flipped.csv
Guardado: saved_archives/maps/realistic/non_flipped/flipped/total_map_realistic_flipped.csv
Guardado: saved_archives/maps/realistic/non_flipped/flipped/uncoated_pmt_map_realistic_flipped.csv
Guardado: saved_archives/maps/realistic/non_flipped/flipped/vis_map_realistic_flipped.csv
Guardado: saved_archives/maps/realistic/non_flipped/flipped/vuv_map_realistic_flipped.csv
Guardado: saved_archives/maps/realistic/non_flipped/flipped/xarap_map_realistic_flipped.csv
Guardado: saved_archives/maps/realistic/non_flipped/flipped/xarap_vis_map_realistic_flipped.csv
Guardado: saved_archives/maps/realistic/non_flipped/flipped/xarap_vuv_map_realistic_flipped.csv
